GSE169159: blood, healthy afaik, longitudinal 
GSE171964: Single-cell data, deseased, PBMC

Preprocessing bulk: 
- outlier detection with PCA (3 samples removed as they were more than 1.5sd away)
raw counts available 
- RLE > 0.6 removed 
- normalised by DESeq2 / BTMsT8

Relative log expression (RLE) plots were generated with EDAseq40; samples with an RLE > 0.6 were removed from the analysis. Differential gene analysis was performed using DESeq241 (v.1.26.0), incorporating participant identifier into the model to account for inter-participant bias. Genes were ranked by the Wald statistic as reported by DESeq2 for GSEA using the BTMs18. Per-participant fold changes were computed by dividing the DESeq2 normalized expression data for the day of interest by either day 0 (for day 1, day2 and day 7) or day 21 (for day 22, 28 and 42). To obtain BTM correlates with age, the age of each participant was compared against the per-participant fold changes for day 22. The resulting correlation values were ranked by t-statistic and analysed with GSEA. The same method was employed to obtain BTM correlates with IFNγ. IFN scores were computed by taking the per-participant mean fold change on day 22 of the unique set of genes present in the 5 interferon BTMs (M75, M111.1, M150, M127 and M68) that significantly correlated with day-22 IFNγ fold change. Similarly, the per-participant M16 gene score was computed using average fold change on day 22 of the genes present in M16.


Check: GSE97590 -> normalized count matrix from GEO used... -> we see this paper also does not compare microarray data with RNseq
(Baseline normalized log2-transformed fold changes were then computed per subject for all genes. GSEA was then performed to identify enriched BTMs using gene lists for each dataset ranked by t-statistic from two-sided Student’s t-tests on the post-vaccination log2-transformed fold changes.)




In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.io import mmread


In [3]:
#With his I mostly want to find out how i can integrate this papers data into ImmPort
# TODO find out what is _BL (i suspect Baseline)

In [5]:
## Testwise integration of 2020_Arachandra_Wimmers
root_folder = '/var/datasets/single_papers/2021_Arunachalam_vaccine/'

metadata_df = pd.read_csv(f'{root_folder}/metadata.txt')
print(metadata_df.shape)
metadata_df.head()

(185, 29)


,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,...,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,sex,source_name,SRA Study
0,SRR18038247,30,RNA-Seq,101,3816732127,PRJNA715457,SAMN25966748,1203925664,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:52:00Z,1,GSM5900628,female,blood,SRP311216
1,SRR18038248,30,RNA-Seq,101,4274587852,PRJNA715457,SAMN25966749,1351097515,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:50:00Z,1,GSM5900629,female,blood,SRP311216
2,SRR18038249,30,RNA-Seq,101,4188481110,PRJNA715457,SAMN25966750,1314161957,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:49:00Z,1,GSM5900630,female,blood,SRP311216
3,SRR18038250,30,RNA-Seq,101,4373580376,PRJNA715457,SAMN25966751,1370039906,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:49:00Z,1,GSM5900631,female,blood,SRP311216
4,SRR18038251,30,RNA-Seq,101,3491091866,PRJNA715457,SAMN25966752,1095712135,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:46:00Z,1,GSM5900632,female,blood,SRP311216


In [6]:
counts_df = pd.read_csv(f'{root_folder}/GSE169159_raw_counts_with_genenames.csv', index_col='gene')
print(counts_df.shape)
counts_df.head()

(23640, 185)


,PID2053_BL,PID2053_Day1.2,PID2053_Day7,PID2053_Day21,PID2053_Day22.23,PID2053_Day28,PID2028_BL,PID2028_Day1.2,PID2028_Day7,PID2028_Day21,...,PID2001_Day7,PID2001_Day21,PID2001_Day22.23,PID2001_Day28,PID2046_BL,PID2046_Day1.2,PID2046_Day7,PID2046_Day21,PID2046_Day22.23,PID2046_Day28
gene,,,,,,,,,,,,,,,,,,,,,
FGR,16312,20268,13433,15317,18685,19379,11275,22251,939,11231,...,33607,21610,37503,39622,30299,19013,56015,16061,23595,23312
STPG1,137,150,154,113,136,179,92,108,105,126,...,86,55,62,63,92,111,246,60,117,97
NIPAL3,2247,2084,1808,1692,1576,2349,1223,1245,735,1197,...,1404,844,1023,782,847,1162,1903,781,1241,1239
KDM1A,1148,1047,876,813,1049,1133,773,778,167,620,...,1270,676,605,695,1019,988,1748,698,1306,930
E2F2,739,705,1312,1290,1878,1111,1189,1254,282,763,...,1302,719,255,677,1150,1197,3270,1622,1226,2080


In [ ]:
# Problem: SRA Toolkit metadata does not have the study Subject accessions ...  Let´s see how else i can get them 
